### https://docs.juliacn.com/latest/manual/metaprogramming/#man-macros

宏提供了一种机制，可以将生成的代码包含在程序的最终主体中。 宏将一组参数映射到返回的 表达式，并且**生成的表达式被直接编译，而不需要运行时 eval 调用**。 宏参数可能包括表达式、字面量和符号。

In [1]:
macro sayhello()
    return :( println("Hello, world!") )
end
@sayhello

Hello, world!


宏在Julia的语法中有一个专门的字符 @ (at-sign)，紧接着是其使用macro NAME ... end 形式来声明的唯一的宏名。在这个例子中，**编译器会把所有的 @sayhello 替换成**：
```julia
:( println("Hello, world!") )
```
**当调用运行 @sayhello 时，解释器立即执行**，因此我们**只会看到计算后的结果：**
```julia
@sayhello()
Hello, world!
```

In [2]:
macro sayhello(name)
    show(name)
    println("\n")
    return :( println("Hello, ", $name) ) 
    # $插值, 把name的值插入进来, 见相关章节
end
@sayhello("human")  # 无括号也可：@sayhello "human"

"human"

Hello, human


我们可以使用函数 macroexpand 查看引用的返回表达式（重要提示： 这是一个非常有用的调试宏的工具）：

In [3]:
ex = macroexpand(Main, :(@sayhello("human")) )
# @sayhello("human")是生成一个表达式 并执行, 
# 外套:(...)就是再看 @sayhello("human") 这个语句的表达式
ex |> display
dump(ex)

:(Main.println("Hello, ", "human"))

"human"

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: GlobalRef
      mod: Module Main
      name: Symbol println
      binding: Core.Binding
        value: #undef
        globalref: GlobalRef#= circular reference @-2 =#
        owner: Core.Binding
          value: println (function of type typeof(println))
          globalref: GlobalRef
            mod: Module Base
            name: Symbol println
            binding: Core.Binding#= circular reference @-2 =#
          owner: Core.Binding#= circular reference @-1 =#
          ty: #undef
          flags: UInt8 0x07
        ty: #undef
        flags: UInt8 0x00
    2: String "Hello, "
    3: String "human"


还有一个宏 **@macroexpand，比 macroexpand 函数更方便：**

In [4]:
@macroexpand @sayhello "human"

"human"



:(Main.println("Hello, ", "human"))

宏是必需的，因为它们在解析代码时执行，因此，**宏允许程序员在运行完整程序之前生成定制代码的片段。** 为了说明差异，请考虑以下示例：

In [5]:
macro twostep(arg)
    println("I execute at parse time. The argument is: ", arg)
    return :(println("I execute at runtime. The argument is: ", $arg))
end

@twostep (macro with 1 method)

In [6]:
twostep_ex = @macroexpand @twostep :(1, 2, 3)

I execute at parse time. The argument is: :((1, 2, 3))


:(Main.println("I execute at runtime. The argument is: ", $(Expr(:copyast, :($(QuoteNode(:((1, 2, 3)))))))))

In [8]:
eval(twostep_ex)

I execute at runtime. The argument is: (1, 2, 3)


In [9]:
@twostep :(1, 2, 3)

I execute at parse time. The argument is: :((1, 2, 3))
I execute at runtime. The argument is: (1, 2, 3)


In [35]:
@twostep (1, 2, 3)

I execute at parse time. The argument is: (1, 2, 3)
I execute at runtime. The argument is: (1, 2, 3)


### https://riptutorial.com/julia-lang/example/6355/reimplementing-the--show-macro

In [1]:
@show 1 + 1

1 + 1 = 2


2

In [2]:
macro myshow(expression)
    quote
        value = $expression
        println($(Meta.quot(expression)), " = ", value)
        value
    end
end

@myshow (macro with 1 method)

In [5]:
Base.remove_linenums!(@macroexpand @myshow 1 + 1)

quote
    var"#64#value" = 1 + 1
    Main.println($(Expr(:copyast, :($(QuoteNode(:(1 + 1)))))), " = ", var"#64#value")
    var"#64#value"
end

In [ ]:
Main.println(

    $(Expr(:copyast, 
           :(
             $(QuoteNode(:(1 + 1)))
            )
          )
     ), 

     " = ", 
     
     var"#64#value"
)

In [6]:
@myshow 1 + 1

1 + 1 = 2


2

### https://discourse.julialang.org/t/macro-hygiene-for-macro-m-ex/80859
Macro hygiene for macro m(ex…)

In [15]:
macro m1(ex)
    :($ex)
end

macro m2(ex...)
    :($ex)
end

@m2 (macro with 1 method)

In [16]:
@macroexpand @m1 a, b

:((Main.a, Main.b))

In [17]:
@macroexpand @m2 a, b

(:((a, b)),)

My question is why the symbols `:a` and `:b` are not qualified with module name in the case of `@m2` unlike to `@m1`?
我的问题是为什么符号 `:a` 和 `:b` 在 `@m2` 不同于 `@m1` 的情况下不具有模块名称？

Ah now I get it, you’re inserting a Tuple into the AST in `@m2`, and **this doesn’t go through macro hygiene because it is not an Expr or Symbol**. You can embed anything you want in the AST in case you didn’t know, but it’s uncommon to do so.<br>

啊，现在我明白了，你在AST中插入了一个元组 `@m2` ，这没有经过宏卫生，因为它不是Expr或Symbol。您可以在AST中嵌入任何您想要的内容（以防您不知道），但这样做并不常见。

In [22]:
dump(@macroexpand @m1 a, b)

Expr
  head: Symbol tuple
  args: Array{Any}((2,))
    1: GlobalRef
      mod: Module Main
      name: Symbol a
      binding: Core.Binding
        value: Array{Vector{Int64}}((3,))
          1: Array{Int64}((2,)) [1, 1]
          2: Array{Int64}((0,)) Int64[]
          3: Array{Int64}((0,)) Int64[]
        globalref: GlobalRef#= circular reference @-2 =#
        owner: Core.Binding#= circular reference @-1 =#
        ty: Any
        flags: UInt8 0x00
    2: GlobalRef
      mod: Module Main
      name: Symbol b
      binding: Core.Binding
        value: Array{Vector{Int64}}((3,))
          1: Array{Int64}((2,)) [1, 1]
          2: Array{Int64}((2,)) [1, 1]
          3: Array{Int64}((2,)) [1, 1]
        globalref: GlobalRef#= circular reference @-2 =#
        owner: Core.Binding#= circular reference @-1 =#
        ty: Any
        flags: UInt8 0x00


In [23]:
dump(@macroexpand @m2 a, b)

Tuple{Expr}
  1: Expr
    head: Symbol tuple
    args: Array{Any}((2,))
      1: Symbol a
      2: Symbol b


Ah I remembered one example which does this, the regex string macro. It inserts a Regex object into the AST so that it is already compiled at compile time.<br>

啊，我记得一个这样做的例子，regex字符串宏。它将一个Regex对象插入到AST中，以便在编译时已对其进行编译。

In [24]:
dump(@macroexpand r"abc")

Regex
  pattern: String "abc"
  compile_options: UInt32 0x040a0002
  match_options: UInt32 0x40000000
  regex: Ptr{Nothing} @0x00000297fc77f480


### https://riptutorial.com/julia-lang/example/30150/python--dict--json-like-syntax-for--dict--literals-

Julia uses the following syntax for dictionaries:<br>
Julia对字典使用以下语法：<br>
`Dict({k₁ => v₁, k₂ => v₂, …, kₙ₋₁ => vₙ₋₁, kₙ => vₙ)`<br>

While Python and JSON looks like this:<br>
而Python和JSON是这样的：<br>
`{k₁: v₁, k₂: v₂, …, kₙ₋₁: vₙ₋₁, kₙ: vₙ}`<br>

For illustrative purposes we could also use this syntax in Julia and add new semantics to it (`Dict` syntax is the idiomatic way in Julia, which is recommended).<br>
为了便于说明，我们也可以在Julia中使用该语法并为其添加新的语义（ `Dict` 语法是Julia中惯用的方式，这是推荐的）。<br>

First let's see what kind of expression it is:<br>
首先让我们看看这是一种什么样的表达：<br>

In [9]:
Meta.parse("{1:2 , 3: 4}") |>display

Meta.parse("{1:2 , 3: 4}") |> Meta.show_sexpr

:({1:2, 3:4})

(:braces, (:call, :(:), 1, 2), (:call, :(:), 3, 4))

In [11]:
dump(Meta.parse("{1:2 , 3: 4}"))

Expr
  head: Symbol braces
  args: Array{Any}((2,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol :
        2: Int64 1
        3: Int64 2
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol :
        2: Int64 3
        3: Int64 4


In [10]:
Meta.parse("Dict(1 => 2 , 3 => 4)") |> display
Meta.parse("Dict(1 => 2 , 3 => 4)") |> Meta.show_sexpr

:(Dict(1 => 2, 3 => 4))

(:call, :Dict, (:call, :(=>), 1, 2), (:call, :(=>), 3, 4))

In [12]:
dump(Meta.parse("Dict(1 => 2 , 3 => 4)"))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol Dict
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol =>
        2: Int64 1
        3: Int64 2
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol =>
        2: Int64 3
        3: Int64 4


The following macro, while simple, allows to demonstrate such code generation and transformation:<br>
下面的宏虽然简单，但可以演示这样的代码生成和转换：

In [13]:
macro dict(expr)
    # Check the expression has the correct form:
    if expr.head ≠ :braces || any(
        sub_expr.head ≠ :call for sub_expr ∈ expr.args) ||any(
        sub_expr.args[1] ≠ :(:) for sub_expr ∈ expr.args)
        error("syntax: expected `{k₁: v₁, k₂: v₂, …, kₙ₋₁: vₙ₋₁, kₙ: vₙ}`")
    end

    # Create empty `:Dict` expression which will be returned:
    block = Expr(:call, :Dict)    # :(Dict())

    # Append `(key => value)` pairs to the block:
    for pair in expr.args
        k, v = pair.args[2], pair.args[3]
        push!(block.args, :($k => $v))
    end    # :(Dict(k₁ => v₁, k₂ => v₂, …, kₙ₋₁ => vₙ₋₁, kₙ => vₙ))

    # Block is escaped so it can reach variables from it's calling scope:
    return esc(block)
end

@dict (macro with 1 method)

In [14]:
@macroexpand @dict {"a": :b, 'c': 1, :d: 2.0}

:(Dict("a" => :b, 'c' => 1, :d => 2.0))

In [15]:
@dict {"a": :b, 'c': 1, :d: 2.0}

Dict{Any, Any} with 3 entries:
  :d  => 2.0
  'c' => 1
  "a" => :b

In [16]:
@dict {                      
           "string": :b,            
           'c'     : 1,             
           :symbol : π,             
           Function: print,         
           (1:10)  : range(1, 10)   
       }            

Dict{Any, Any} with 5 entries:
  :symbol  => π
  "string" => :b
  1:10     => 1:10
  Function => print
  'c'      => 1

The last example is exactly equivalent to:<br>
最后一个例子完全等同于：

In [17]:
Dict(                      
    "string" => :b,            
    'c'      => 1,             
    :symbol  => π,             
    Function => print,         
    (1:10)   => range(1, 10)   
)

Dict{Any, Any} with 5 entries:
  :symbol  => π
  "string" => :b
  1:10     => 1:10
  Function => print
  'c'      => 1

In [18]:
@dict {"one": 1, "two": 2, "three": 3, "four": 4, "five" => 5}

LoadError: LoadError: syntax: expected `{k₁: v₁, k₂: v₂, …, kₙ₋₁: vₙ₋₁, kₙ: vₙ}`
in expression starting at e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X44sZmlsZQ==.jl:1

### https://juliasnippets.blogspot.com/2018/02/a-small-adventure-into-julia-macro-land.html<br>
**非主流例子，可忽略，也没怎么看懂**

The Julia Manual teaches us that<br>
《茱莉亚手册》告诉我们这一点<br>

*Julia evaluates default values of function arguments every time the method is invoked, unlike in Python where the default values are evaluated only once when the function is defined.* <br>

*Julia 每次调用方法时都会评估函数参数的默认值，这与 Python 不同，在 Python 中，默认值仅在定义函数时评估一次.*<br>

in *Noteworthy differences from Python* section.<br>

However, sometimes you want a value to be evaluated only once when the function is defined. Recently a probably obvious fact has downed on me that this can conveniently be achieved using macros. Here is a simple example:<br>

但是，有时您希望在定义函数时只对一个值求值一次。最近，我发现了一个显而易见的事实，那就是使用宏可以方便地实现这一点。这里有一个简单的例子：

In [1]:
macro intvec()
    println("Hey!")
    Int[]
end

function f(x)
    v = @intvec()
    push!(v, x)
    v
end

Hey!


f (generic function with 1 method)

In [2]:
@macroexpand @intvec

Hey!


Int64[]

When you run this code you can observe that `Hey!` is printed once (when `@intvec` is evaluated).<br>
当你运行此代码时，你可以观察到`Hey！`被打印一次（当评估`@intvec`时）<br>

Now let us check how the function works. Running:<br>
现在让我们看看这个函数是如何工作的。运行:

In [3]:
for i in 1:5
    println(i)
    println(f(i))
    println(v)
end

1
[1]


UndefVarError: UndefVarError: `v` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [4]:
for i in 1:5    # 上个cell已生成[],所以这里起始 [1,1]
    println(f(i))  
end

[1, 1]
[1, 1, 2]
[1, 1, 2, 3]
[1, 1, 2, 3, 4]
[1, 1, 2, 3, 4, 5]


and we can see that `@intvec` was not run (no `Hey!` is printed). This is natural - macros are evaluated only once before the program is actually executed.<br>
我们可以看到 `@intvec` 没有运行（没有打印 `Hey!`）。这是很自然的 - 在程序实际执行之前，宏只被评估一次。<br>

Another small example using comprehensions:
另一个使用推导式的小例子：

In [6]:
a = [Int[] for i in 1:3]
b = [@intvec() for i in 1:3]

a |>display
b |>display

3-element Vector{Vector{Int64}}:
 []
 []
 []

3-element Vector{Vector{Int64}}:
 []
 []
 []

Hey!


In [7]:
push!(a[1], 1)
push!(b[1], 1)

a |>display
b |>display

3-element Vector{Vector{Int64}}:
 [1]
 []
 []

3-element Vector{Vector{Int64}}:
 [1]
 [1]
 [1]

In [8]:
push!(a[1], 1)
push!(b[1], 1)

a |>display
b |>display

3-element Vector{Vector{Int64}}:
 [1, 1]
 []
 []

3-element Vector{Vector{Int64}}:
 [1, 1]
 [1, 1]
 [1, 1]

And we see that in case of  `b` each index points to the same array.<br>
我们看到，在 `b` 的情况下，每个索引都指向同一个数组。<br>

One might ask if it is only a special case or it does actually mater in daily Julia usage. The situation where this distinction is important came up recently when writing documentation of `@threads` macro. If you check out a definition of `f_fix` function there you will find:<br>

有人可能会问，这是否只是一种特殊情况，还是在 Julia 的日常使用中确实很重要。最近在编写 `@threads` 宏的文档时，出现了这种区别很重要的情况。如果您查看 `f_fix` 函数的定义，您会发现：

In [23]:
using Base.Threads
function f_fix()
    s = repeat(["123", "213", "231"], outer=1000)
    x = similar(s, Int)
    rx = [Regex("1") for i in 1:nthreads()]
    @threads for i in 1:3000
        x[i] = findfirst(rx[threadid()], s[i]).start
    end
    count(v -> v == 1, x)
end
f_fix()

1000

where we use `Regex("1")` instead of a more natural `r"1"` exactly because the latter would create only one instance of regex object.<br>
这里我们使用 `Regex("1")` 而不是更自然的 `r"1"`，因为后者只会创建一个 regex 对象实例。<br>

So the question is what is the benefit of `r"1"` then? The answer is performance - we have to compile the regex only once. This saves time if a function containing it would be called many times, e.g.:<br>
那么问题是 `r"1"` 有什么好处呢？答案是性能 - 我们只需编译一次 regex。如果包含它的函数被多次调用，这可以节省时间，例如：

In [17]:
match(r"1", "123")

RegexMatch("1")

In [18]:
match(Regex("1"), "123")

RegexMatch("1")

In [8]:
f() = match(r"1", "123")
g() = match(Regex("1"), "123")

g (generic function with 1 method)

In [11]:
using BenchmarkTools

In [12]:
@benchmark f()

BenchmarkTools.Trial: 10000 samples with 878 evaluations.
 Range (min … max):  133.144 ns …   6.396 μs  ┊ GC (min … max): 0.00% … 97.12%
 Time  (median):     140.205 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   159.762 ns ± 237.093 ns  ┊ GC (mean ± σ):  8.05% ±  5.39%

  ▁▆█▆▄▂▁▃▃▂▁▁▁▁▁▁                                              ▂
  █████████████████▇▇▇▇▇▆▆▇▆▆▇▅▇▆▆▆▆▆▆▅▅▅▆▅▅▄▅▄▅▅▅▅▃▅▆▅▃▁▅▄▃▅▄▅ █
  133 ns        Histogram: log(frequency) by time        293 ns <

 Memory estimate: 128 bytes, allocs estimate: 3.

In [13]:
@benchmark g()

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   4.429 μs … 327.000 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.057 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.154 μs ±   6.463 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂█▅▁▁ ▁▂    ▁    ▂  ▂▂▇   ▃▁   █ ▁▂▁   ▃                      
  █████▆██▆▅▆██▄██▇█▇█████▃▇██▂███▇█████▇██▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▄
  4.43 μs         Histogram: frequency by time         25.8 μs <

 Memory estimate: 160 bytes, allocs estimate: 4.


FrontRangeGamer February 26, 2018 at 11:54 PM<br>

The manual says "A macro maps a tuple of arguments to a returned expression...". I found your results a bit confusing until I realized `@intvec` isn't returning an expression. If you use `macro intvec2() quote Int[] end end` you get a distinct array on each invocation. Why do repeated calls to `push!(@intvec,1)` always return `[1]` and not return Vectors of increasing size? I expect the latter from your `b = [@intvec() for i in 1:3]` example.<br>

手册上说“宏将参数元组映射到返回的表达式…”。我发现你的结果有点混乱，直到我意识到`@intvec`不返回表达式。如果使用`macro intvec2() quote Int[] end end`，每次调用都会得到一个不同的数组。为什么重复调用`push!(@intvec,1)`总是返回`[1]`而不返回大小不断增加的向量？我希望后者从你的`b = [@intvec() for i in 1:3]`的例子。

In [ ]:
push!(@intvec,1)

Hey!


1-element Vector{Int64}:
 1

In [ ]:
push!(@intvec,1)

Hey!


1-element Vector{Int64}:
 1

Replies
Bogumił KamińskiFebruary 27, 2018 at 12:23 AM <br>

This is exactly the point - macro does not have to return an expression. And, for instance, this is what regex macro does. The reason why repeated calls to `push!(@intvec,1)` return `[1]` is that the macro is invoked every time you make this call, but if you would write for example `for i in 1:5 global x = push!(@intvec, i) end` at the end `x` will contain five elements as `@intvec` is invoked only once when the loop is parsed.<br>

这正是关键所在——宏不必返回表达式。例如，这就是regex宏所做的。重复调用 `push!(@intvec,1)`  return `[1]` 的原因是每次调用时都会调用宏，但是如果您编写例如`for i in 1:5 global x = push!(@intvec, i) end` at the end `x` 将包含五个元素，**因为 `@intvec` 在解析循环时只调用一次**。

In [ ]:
for i in 1:5
    global x = push!(@intvec, i) 
end
x

Hey!


5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [ ]:
for i in 1:5
    global x = push!(@intvec, i) 
end
x

Hey!


5-element Vector{Int64}:
 1
 2
 3
 4
 5